In [41]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset
import time
from tqdm import tqdm

### Information
- We will do a few preliminary exercises and also build a character level MLP language model.
- This model will be similar to the model we did in class, except that we will have characters as tokens, not words.
- You will need a conda environment for this, here is general information on this.
 - https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html
 - PyTorch: https://anaconda.org/pytorch/pytorch
 
In the code below, FILL-IN the code necessary in the hint string provided.

### Preliminary exercises
- Please fill in the cells below with the asked for data.

In [42]:
torch.manual_seed(1)

In [43]:
# Create an embedding layer for a vocabulary of size 10 and the word vectors are each of dimension 5.
e = nn.Embedding(10, 5)

# Extract the embedding for the word whose token index is 3. What is the shape of this vector?
v = e(torch.tensor(3))


# Extract the weight matrix from the layer e.
# Create a linear layer (with no bias) of size 10 by 5 and set it's data to the embedding matrix.
l = nn.Linear(5, 10, bias=False)
l.weight = e.weight

# Insert inside of the assert below some sort of equality check between l.weight and e.weight; it should pass to true.
# Hint: look up torch.all() and torch.eq()

assert(torch.eq(l.weight, e.weight).all())

In [44]:
# Create a batch of size 2 with entries [0, 1, 2] and [2, 3, 4] in the data batch.
x = torch.tensor([[0, 1, 2], [2, 3, 4]])

In [45]:
# What is the dimesion of this batch ran through the embeding layer?
assert(e(x).shape == torch.Size([2, 3, 5]))

### Constants and configs used below.

In [46]:
DEVICE = "cpu"
LR = 4.0
BATCH_SIZE = 16
NUM_EPOCHS = 5
MARKER = '.'
# N-gram level; P(w_t | w_{t-1}, ..., w_{t-n+1}).
# We use 3 words to predict the next word.
n = 4
# Hidden layer dimension.
h = 20
# Word embedding dimension.
m = 20

### Get the dataset and the tokenizer.

In [47]:
class CharDataset(Dataset):
    def __init__(self, words, chars):
        self.words = words
        self.chars = chars
        # Inverse dictionaries mapping char tokens to unique ids and the reverse.
        # Tokens in this case are the unique chars we passed in above.
        # Each token should be mappend to a unique integer and MARKER should have token 0.
        # For example, stoi should be like {'.' -> 0, 'a' -> 1, 'b' -> 2} if I pass in chars = '.ab'.
        dic_stoi = {}
        dic_itos = {}
        
        for i in range(len(chars)):
            dic_stoi[chars[i]] = i
            dic_itos[i] = chars[i]
            
        self.stoi = dic_stoi
        self.itos = dic_itos

    def __len__(self):
        # Number of words.
        return len(self.words)

    def contains(self, word):
        # Check if word is in self.words and return True/False if it is, is not.
        return word in self.words

    def get_vocab_size(self):
        # Return the vocabulary size.
        return len(self.chars)

    def encode(self, word):
        # Express this word as a list of int ids. For example, maybe ".abc" -> [0, 1, 2, 3].
        # This assumes 'a' -> 1, etc.
        return [self.stoi[char] for char in word]
    
    def decode(self, tokens):
        # For a set of tokens, return back the string.
        # For example, maybe [1, 1, 2] -> "aac"
        return [self.itos[token] for token in tokens]

    def __getitem__(self, idx):
        # This is used so we can loop over the data.
        word = self.words[idx]
        return self.encode(word)

In [48]:
def create_datasets(window, input_file = 'names.txt'):
    """
    This takes a file of words and separates all the words.
    It then gets all the characters present in the universe of words and then ouputs the statistics. 
    """
    with open(input_file, 'r') as f:
        data = f.read()
    # Split the file by new lines. You should get a list of names.
    words = data.split('\n')
                       
    words = [word.replace(' ', '') for word in words]# This gets rid of any trailing and starting white spaces.
    words = [i for i in words if i] # Filter out all the empty words.
    
    # This gets the universe of all characters.
    chars = sorted(list(set([char for word in words for char in word])))
    
    # Will force chars to have MARKER having index 0.
    chars= [MARKER] + chars
    
    # Pad each word with a context window of size n-1.
    # Why? a word like "abc" should becomes "..abc.." if the window is size 3.
    # This is some we can get pair of (x, y) data like this: ".." -> "a", ".a" -> "b", "ab" -> "c", "bc" -> ".", "c." -> "."
    # I.e. this allows us to know that "a" is a start character.
    # So you should get something like ["ab", "c"] -> ["..ab..", "..c.."], for example.
    words  = [MARKER * (window - 1) + word + MARKER * (window - 1) for word in words]
            
    print(f"The number of examples in the dataset: {len(words)}")
    print(f"The number of unique characters in the vocabulary: {len(chars)}")
    print(f"The vocabulary we have is: {''.join(chars)}")

    # Partition the input data into a training, validation, and the test set.
    out_of_sample_set_size = min(2000, int(len(words) * 0.1)) # We use 10% of the training set, or up to 2000 examples.
    test_set_size = 1500
    
    # First, get a random permutation of randomly permute of size len(words).
    # Then, convert this to a list. 
    # This index list is used below to get the train, validation, and test sets.
    rp = torch.randperm(len(words)).tolist()
    
    # Get train, validation, and test set.
    train_words = [words[i] for i in rp[:-out_of_sample_set_size]]
    validation_words = [words[i] for i in rp[-out_of_sample_set_size:-test_set_size]]
    test_words = [words[i] for i in rp[-test_set_size:]]    
    
    print(f"We've split up the dataset into {len(train_words)}, {len(validation_words)}, {len(test_words)} training, validation, and test examples")

    # But the data in the data set objects.
    train_dataset = CharDataset(train_words, chars)
    validation_dataset = CharDataset(validation_words, chars)
    test_dataset = CharDataset(test_words, chars)

    return train_dataset, validation_dataset, test_dataset

In [49]:
train_dataset, validation_dataset, test_dataset = create_datasets(n)

The number of examples in the dataset: 32033
The number of unique characters in the vocabulary: 27
The vocabulary we have is: .abcdefghijklmnopqrstuvwxyz
We've split up the dataset into 30033, 500, 1500 training, validation, and test examples


## Explore the data

In [62]:
# Get the first word in "train_dataset"
train_dataset[0]

[0, 0, 0, 14, 9, 25, 1, 13, 0, 0, 0]

In [51]:
# Get the stoi map of train_dataset. How many keys does it have?
len(train_dataset.stoi)

27

### Get the dataloader

In [52]:
def create_dataloader(dataset, window):
    x_list = []
    y_list = []
    # For ech word.
    for i, word in enumerate(dataset):
        # Grab a context of size window and window-1 characters will be in x, 1 will be in y.
        for j, _ in enumerate(word):
            # If there is no widow of size window left, break.
            if j + window > len(word) - 1:
                break
            word_window = word[j:j+window]
            x, y = word_window[:window-1],word_window[-1]
            x_list.append(x)
            y_list.append(y)
            
    return DataLoader(
        TensorDataset(torch.tensor(x_list), torch.tensor(y_list)),
        BATCH_SIZE,
        shuffle=True
    )

In [53]:
train_dataloader = create_dataloader(train_dataset, n)
validation_dataloader = create_dataloader(validation_dataset, n)
test_dataloader = create_dataloader(test_dataset, n)

### Set up the model
- Identical to lecture. Please look over that!

In [54]:
# One of the first Neural language models!
class CharacterNeuralLanguageModel(nn.Module):
    def __init__(self, V, m, h, n):
        super(CharacterNeuralLanguageModel, self).__init__()
        
        # Vocabulary size.
        self.V = V
        
        # Embedding dimension, per word.
        self.m = m
        
        # Hidden dimension.
        self.h = h
        
        # N in "N-gram"
        self.n = n
        
        # Can you change all this stuff to use nn.Linear?
        # Ca also use nn.Parameter(torch.zeros(V, m)) for self.C but then we need one-hot and this is slow.
        self.C = nn.Embedding(V, m)
        self.H = nn.Parameter(torch.zeros((n-1) * m, h))
        self.W = nn.Parameter(torch.zeros((n-1) * m, V))
        self.U = nn.Parameter(torch.zeros(h, V))
        
        self.b = torch.nn.Parameter(torch.ones(V))
        self.d = torch.nn.Parameter(torch.ones(h))
        
        self.init_weights()

    def init_weights(self):
        # Intitialize C, H, W, U in a nice way. Use xavier initialization for the weights.
        # On a first run, just pass.
        with torch.no_grad():
            torch.nn.init.xavier_uniform_(self.C.weight)
            torch.nn.init.xavier_uniform_(self.H)
            torch.nn.init.xavier_uniform_(self.W)
            torch.nn.init.xavier_uniform_(self.U)
        
        
    def forward(self, x):
        
        # x is of dimenson N = batch size X n-1
        
        # N X (n-1) X m 
        x = self.C(x)
        
        # N
        N = x.shape[0]
        
        # N X (n-1) * m
        x = x.view(N,-1)
    
        # N X V
        y = self.b + torch.matmul(x, self.W) + torch.matmul(nn.Tanh()(self.d + torch.matmul(x, self.H)), self.U)
        
        return y

### Set up the model.

In [55]:
# Identical to lecture.
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
model = CharacterNeuralLanguageModel(
    train_dataset.get_vocab_size(), m, h, n
).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

In [56]:
# How many parameters does the neural network have?
# Hint: look up model.named_parameters and the method "nelement" on a tensor.
# See also the XOR notebook where we count the gradients that are 0.
# There, we loop over the parameters.
number_parameters = 0
for name, param in model.named_parameters():
    number_parameters += 1
    print(name, param.shape, param.requires_grad)
print(f"Number of parameters: {number_parameters}")


H torch.Size([60, 20]) True
W torch.Size([60, 27]) True
U torch.Size([20, 27]) True
b torch.Size([27]) True
d torch.Size([20]) True
C.weight torch.Size([27, 20]) True
Number of parameters: 6


### Train the model.

In [57]:
def calculate_perplexity(total_loss, total_batches):
    return torch.exp(torch.tensor(total_loss / total_batches)).item()

In [58]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_loss, total_batches = 0.0, 0.0
    log_interval = 500

    for idx, (x, y) in tqdm(enumerate(dataloader)):
        optimizer.zero_grad()
        
        logits = model(x)
                        
        # Get the loss.
        loss = criterion(input=logits, target=y.view(-1))

        # Do back propagation.
        loss.backward()
                        
        # Clip the gradients so they don't explode. Look at how this is done in lecture.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        
        # Do an optimization step.
        optimizer.step()
        
        total_loss += loss.item()
        total_batches += 1
                
        if idx % log_interval == 0 and idx > 0:
            perplexity = calculate_perplexity(total_loss,  total_batches)
            print(
                "| epoch {:3d} "
                "| {:5d}/{:5d} batches "
                "| perplexity {:8.3f} "
                "| loss {:8.3f} "
                .format(
                    epoch,
                    idx,
                    len(dataloader),
                    perplexity,
                    total_loss / total_batches,
                )
            )
            total_loss, total_batches = 0.0, 0

In [59]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_loss, total_batches = 0.0, 0

    with torch.no_grad():
        for idx, (x, y) in enumerate(dataloader):
            logits = model(x)
            total_loss += criterion(input=logits, target=y.squeeze(-1)).item()
            total_batches += 1
    return total_loss / total_batches, calculate_perplexity(total_loss,  total_batches)

In [60]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    loss_val, perplexity_val = evaluate(validation_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} "
        "| time: {:5.2f}s "
        "| valid perplexity {:8.3f} "
        "| valid loss {:8.3f}".format(
            epoch,
            time.time() - epoch_start_time,
            perplexity_val,
            loss_val
        )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
loss_test, perplexity_test = evaluate(test_dataloader, model, criterion)
print("test perplexity {:8.3f} | test loss {:8.3f} ".format(perplexity_test, loss_test))

1000it [00:00, 2518.92it/s]

| epoch   1 |   500/15247 batches | perplexity    9.705 | loss    2.273 
| epoch   1 |  1000/15247 batches | perplexity    8.738 | loss    2.168 


1779it [00:00, 2581.16it/s]

| epoch   1 |  1500/15247 batches | perplexity    8.605 | loss    2.152 
| epoch   1 |  2000/15247 batches | perplexity    8.323 | loss    2.119 


2823it [00:01, 2396.74it/s]

...

| epoch   5 | 13500/15247 batches | perplexity    7.042 | loss    1.952 
| epoch   5 | 14000/15247 batches | perplexity    7.005 | loss    1.947 


14978it [00:06, 2483.20it/s]

| epoch   5 | 14500/15247 batches | perplexity    7.007 | loss    1.947 


15247it [00:06, 2486.01it/s]


| epoch   5 | 15000/15247 batches | perplexity    6.977 | loss    1.943 
-----------------------------------------------------------
| end of epoch   5 | time:  6.16s | valid perplexity    6.974 | valid loss    1.942
-----------------------------------------------------------
Checking the results of test dataset.
test perplexity    7.121 | test loss    1.963 


Hint: For the above, you should see your loss around 2.0 and going down. Similarly to perplexity which should be aroud 7 to 8.

In [65]:
3 * [train_dataset.stoi[MARKER]]

[0, 0, 0]

## Generate some text.

In [88]:


def generate_word(model, dataset, window):
    generated_word = []
    # Set the context to a window-1 length array having just the MARKER character's token_id.
    context = (window - 1) * [dataset.stoi[MARKER]]
    
    while True:
        logits = model(torch.tensor(context).view(1, -1))
        
        # Get the probabilities from the logits.
        # Hint: softmax!
        probs =  nn.Softmax(dim=1)(logits)
        
        # Get 1 sample from a multinomial having the above probabilities.
        token_id = torch.multinomial(probs, 1).item()
        
        # Append the token_id to the generated word.
        generated_word.append(token_id)
        
        # Move the context over 1, drop the first (oldest) token and apped the new one above.
        # The size of the resulting context should be the same.
        # For exaple, if it was "[0, 1, 2]" and you generated 4, it should now be [1, 2, 4].
        context = context[1:] + [token_id]
        
        if token_id == 0:
            # If you generate token_id = 0, i.e. '.', break out.
            break
    # Return and decode the generated word to a string.        
    return ''.join(dataset.decode(generated_word))

In [89]:
torch.manual_seed(1)
for _ in range(50):
    print(generate_word(model, train_dataset, n))

ama.
ele.
lia.
aldi.
jarorsse.
dez.
bria.
jairestlei.
revy.
madlais.
hoanna.
dacelian.
alalie.
shais.
maya.
jouston.
zailah.
ede.
rie.
gros.
aukh.
bamaka.
anyaarius.
kelee.
har.
jami.
naekshreem.
kaylen.
quyla.
naygusen.
mayanatram.
ahazoriexsunya.
shamonti.
hori.
evfiah.
rosie.
vaivel.
ynalaydin.
kenasia.
dar.
wun.
jayana.
ris.
nor.
ilyn.
marri.
alevante.
kalyn.
desleeshanaa.
daniellaenimariinilah.
